# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

## Загрузка и подготовка данных

In [2]:
import pandas as pd
import numpy as np
import seaborn as sn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler 
from numpy.random import RandomState

In [3]:
try:
    df_0 = pd.read_csv('/datasets/geo_data_0.csv')
    df_1 = pd.read_csv('/datasets/geo_data_1.csv')
    df_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    df_0 = pd.read_csv('geo_data_0.csv')
    df_1 = pd.read_csv('geo_data_1.csv')
    df_2 = pd.read_csv('geo_data_2.csv')

In [4]:
#смотрим состаяние данных
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
#удаляем уникальные id
df_0 = df_0.drop('id',axis=1)
df_1 = df_1.drop('id',axis=1)
df_2 = df_2.drop('id',axis=1)
df_0

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...
99995,0.971957,0.370953,6.075346,110.744026
99996,1.392429,-0.382606,1.273912,122.346843
99997,1.029585,0.018787,-1.348308,64.375443
99998,0.998163,-0.528582,1.583869,74.040764


In [8]:
# проверяем дубликаты
display(df_0.duplicated().sum())
display(df_1.duplicated().sum())
display(df_2.duplicated().sum())

0

0

0

In [9]:
df_1.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [25]:
df_0.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [26]:
df_2.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


<div class="alert alert-info" role="alert">
 
Данные в хорошем состоянии,пропусков нет,дубликатов нет,типы в порядке
    
Столбец с идентификатором лучше удалить т.к. в нем нет полезной информации для обучения модели.
</div>

## Обучение и проверка модели

In [14]:
import warnings
warnings.filterwarnings("ignore")

In [15]:
X_0 = df_0.drop('product',axis=1)
y_0=df_0['product']
X_train_0,X_time_0,y_train_0,y_time_0 = train_test_split(X_0,y_0,random_state=12345,test_size = 0.4) # разделяем выборки
X_valid_0,X_test_0,y_valid_0,y_test_0 = train_test_split(X_time_0,y_time_0,random_state=12345,test_size=0.5) 
# маштабируем значения в выборках
numeric = ['f0', 'f1', 'f2']
scaler = StandardScaler()
scaler.fit(X_train_0[numeric])
X_train_0[numeric] = scaler.transform(X_train_0[numeric] )
X_valid_0[numeric]  = scaler.transform(X_valid_0[numeric] )
X_test_0[numeric]  = scaler.transform(X_test_0[numeric] )
pd.options.mode.chained_assignment = None
# применим модель линейной регрессии
model_0 =  LinearRegression()
model_0.fit(X_train_0,y_train_0)
pred_0 = model_0.predict(X_valid_0)
#посчитаем среднее и среднюю ошибку предсказаний
rmse = mean_squared_error(y_valid_0,pred_0)  ** 0.5 
print('    pred:',pred_0.mean())
print('    rmse:',rmse)

    pred: 92.50862491249552
    rmse: 37.54151515850603


In [16]:
X_1 = df_1.drop('product',axis=1)
y_1=df_1['product']
X_train_1,X_time_1,y_train_1,y_time_1 = train_test_split(X_1,y_1,random_state=12345,test_size = 0.4)
X_valid_1,X_test_1,y_valid_1,y_test_1 = train_test_split(X_time_1,y_time_1,random_state=12345,test_size=0.5)
numeric = ['f0', 'f1', 'f2']
scaler = StandardScaler()
scaler.fit(X_train_1[numeric])
X_train_1[numeric] = scaler.transform(X_train_1[numeric] )
X_valid_1[numeric]  = scaler.transform(X_valid_1[numeric] )
X_test_1[numeric]  = scaler.transform(X_test_1[numeric] )
pd.options.mode.chained_assignment = None
model_1 =  LinearRegression()
model_1.fit(X_train_1,y_train_1)
pred_1 = model_1.predict(X_valid_1)
rmse = mean_squared_error(y_valid_1,pred_1)  ** 0.5
print('    pred:',pred_1.mean())
print('    rmse:',rmse)

    pred: 68.49008336465833
    rmse: 0.8858505980507573


In [17]:
X_2 = df_2.drop('product',axis=1)
y_2=df_2['product']
X_train_2,X_time_2,y_train_2,y_time_2 = train_test_split(X_2,y_2,random_state=12345,test_size = 0.4)
X_valid_2,X_test_2,y_valid_2,y_test_2 = train_test_split(X_time_2,y_time_2,random_state=12345,test_size=0.5)
numeric = ['f0', 'f1', 'f2']
scaler = StandardScaler()
scaler.fit(X_train_2[numeric])
X_train_2[numeric] = scaler.transform(X_train_2[numeric] )
X_valid_2[numeric]  = scaler.transform(X_valid_2[numeric] )
X_test_2[numeric]  = scaler.transform(X_test_2[numeric] )
pd.options.mode.chained_assignment = None
model_2 =  LinearRegression()
model_2.fit(X_train_2,y_train_2)
pred_2 = model_2.predict(X_valid_2)
rmse = mean_squared_error(y_valid_2,pred_2)  ** 0.5
print('    pred:',pred_2.mean())
print('    rmse:',rmse)

    pred: 95.0007676728214
    rmse: 39.72337541963802


In [27]:
def split_scaler(df):
    X = df.drop(['product'],axis=1)
    y = df['product']
    X_train,X_time_test,y_train,y_time_test = train_test_split(X,y,random_state=12345,test_size = 0.4)
    X_valid,X_test,y_valid,y_test = train_test_split(X_time_test,y_time_test,random_state=12345,test_size = 0.5)
    numeric = ['f0', 'f1', 'f2']
    scaler = StandardScaler()
    scaler.fit(X_train[numeric])
    X_train[numeric] = scaler.transform(X_train[numeric] )
    X_valid[numeric]  = scaler.transform(X_valid[numeric] )
    X_test[numeric]  = scaler.transform(X_test[numeric] )
    pd.options.mode.chained_assignment = None
    model =  LinearRegression()
    model.fit(X_train,y_train)
    pred = model.predict(X_valid)
    rmse = (mean_squared_error(y_valid,pred))  ** 0.5
    print('    pred:',pred.mean())
    print('    rmse:',rmse)
print('df_0')
split_scaler(df_0)
print('df_1')
split_scaler(df_1)
print('df_2')
split_scaler(df_2)

df_0
    pred: 92.50862491249552
    rmse: 37.54151515850603
df_1
    pred: 68.49008336465833
    rmse: 0.8858505980507573
df_2
    pred: 95.0007676728214
    rmse: 39.72337541963802


<div class="alert alert-info" role="alert">
 
На данных региона geo_data_1 модель предсказывает хорошо,для остальных метрика очень слабая
    
Но для определения лучшего региона нужно так-же посмотреть на прибыль и риски
</div>

## Подготовка к расчёту прибыли

In [19]:
BUDGET_PART = 10000000000 / 200 #бюджен на 1 скважену
BUDGET = 10000000000 # общий бюджет
BAR = 450 * 1000 # доход с 1 бареля
PAYBACK = BUDGET_PART / BAR # минимальный обьем скважены для нулевой доходности
print(PAYBACK)

111.11111111111111


In [20]:
print(df_0['product'].mean())
print(df_1['product'].mean())
print(df_2['product'].mean())

92.50000000000001
68.82500000000002
95.00000000000004


### большинство скважин убыточные

In [21]:
#пишем функцию расчета дохода
state = RandomState(12345)
def incomes(target,predict,count):
    target = pd.Series(target)
    target = target.reset_index(drop = True)
    predict = pd.Series(predict)
    part_500 = predict.sample(n = 500,random_state = state, replace = True)
    part_500 = part_500.sort_values(ascending = False).head(count)
    target_part = target[part_500.index]
    income = target_part.sum() * BAR - BUDGET
    return income

## Расчёт прибыли и рисков 

In [22]:
values_0 = []
for i in range(1000):
    value = incomes(y_valid_2,pred_2,200)
    value = value / 1000000
    values_0.append(value)
values_0 = pd.Series(values_0)
mean = values_0.mean()
up = values_0.quantile(0.975)
low = values_0.quantile(0.025)
print('mean:',mean)
print('up:',up)
print('low:',low)
print('Вероятность убытка:', (values_0 < 0).mean() * 100 , '%')

mean: 412.6784721392619
up: 909.5927907701544
low: -109.87550806414032
Вероятность убытка: 6.1 %


In [23]:
values_1 = []
for i in range(1000):
    value = incomes(y_valid_1,pred_1,200)
    value = value / 1000000
    values_1.append(value)
values_1 = pd.Series(values_1)
mean = values_1.mean()
up = values_1.quantile(0.975)
low = values_1.quantile(0.025)
print('mean:',mean)
print('up:',up)
print('low:',low)
print('Вероятность убытка:', (values_1 < 0).mean() * 100 , '%')

mean: 452.06798625523527
up: 845.2328827952016
low: 60.25388349177985
Вероятность убытка: 0.8 %


In [24]:
values_2 = []
for i in range(1000):
    value = incomes(y_valid_0,pred_0,200)
    value = value / 1000000
    values_2.append(value)
values_2 = pd.Series(values_2)
mean = values_2.mean()
up = values_2.quantile(0.975)
low = values_2.quantile(0.025)
print('mean:',mean)
print('up:',up)
print('low:',low)
print('Вероятность убытка:', (values_2 < 0).mean() * 100 , '%')

mean: 364.5397797892293
up: 868.7869152884986
low: -120.00971730032187
Вероятность убытка: 7.000000000000001 %


<div class="alert alert-info" role="alert">
Перспективнее регион geo_data_1
    
    
Минимальный риск (минимальная прибыль в 95% доверительном интервале:60млн)

Лучшие средние по запасам сырья в скважинах (средняя прибыль 425млн)
</div>

## Чек-лист готовности проекта